In [1]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_isosurface
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [2]:
def plot_PV(level):
    print(f"Creating the {level}-hPa PV Map")
    ilev = list(lev.m).index(level*100.)

    uwnd_ilev = uwnd[ilev].metpy.convert_units('kt')
    vwnd_ilev = vwnd[ilev].metpy.convert_units('kt')
    
    sped_ilev = mpcalc.wind_speed(uwnd_ilev, vwnd_ilev)
    div_ilev = mpcalc.smooth_n_point(div[ilev], 9, 2)
    
    epv_smooth = mpcalc.smooth_n_point(epv[ilev], 9, 2)

    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, sped_ilev, range(10,230,20), cmap=plt.cm.BuPu)
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2,15,1), colors='black')
    plt.clabel(cs, fmt='%d')
    
    cs2 = ax.contour(clons, clats, div_ilev*1e5, range(1,50,3), colors='grey', linestyles='dashed')
    plt.clabel(cs2, fmt='%d')

    plt.title(f'{int(lev[ilev].m/100)}-hPa PV (PVU)'', Divergence ($*10^5$ $s^{-1}$), and Wind Spped (kt)', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')

    plt.savefig(f'{int(lev[ilev].m/100)}-hPa_PV_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    #plt.close()

In [3]:
def plot_DT():
    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, thta_DT, np.arange(258,427,6), cmap=plt.cm.coolwarm, extend='both')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, mpcalc.smooth_n_point(relvor_925850, 9, 2)*1e4, np.arange(0.5,30,.5), colors='black')
    plt.clabel(cs, fmt='%d')

    cs2 = ax.contour(clons, clats, thta_DT, np.arange(258,427,6), colors='grey', linestyles='dotted')
    #plt.clabel(cs2, fmt='%d')

    ax.barbs(lons[wind_slice], lats[wind_slice],
             uwnd_DT[wind_slice], vwnd_DT[wind_slice],
             transform=ccrs.PlateCarree())

    plt.title(r'DT Potential Temp (K), 925-850-hPa Avg. Rel. Vor. ($*10^4$ $s^{-1}$),'
              ' and Wind Spped (kt)', loc='left')
    plt.title('Valid Time: {}'.format(vtime), loc='right')

    plt.savefig('DT_Potential_Temp_{0:%Y%m%d_%H}00.png'.format(date), bbox_inches='tight', dpi=150)
    plt.show()
    #plt.close()

In [4]:
date = datetime(2020, 1, 11, 12)

ds = xr.open_dataset('http://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
                     f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2')

# ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))
lev_name = ds.Temperature_isobaric.dims[1]

ip100_3 = list(ds[lev_name].values).index(10000)-1
#ip100_5 = list(ds.isobaric5.values).index(10000)-1

lev = ds[lev_name].values[ip100_3:] * units.Pa

wind_slice = (slice(None, None, 5), slice(None, None, 5))

lats = ds.lat.data
lons = cutil.add_cyclic_point(ds.lon.data)

# Set subset slice for the geographic extent of data to limit download
lon_slice = slice(400,701)
lat_slice = slice(10,160)

lat1d = lats[lat_slice]
lon1d = lons[lon_slice]
lons, lats = np.meshgrid(lon1d, lat1d)

subset = dict(time=date,
              vertical=slice(70 * units.hPa, 400*units.hPa),
              latitude=slice(70, 15, 2),
              longitude=slice(360-130, 360-65, 2))

uwnd = ds['u-component_of_wind_isobaric'].metpy.sel(subset)
vwnd = ds['v-component_of_wind_isobaric'].metpy.sel(subset)
tmpk = ds['Temperature_isobaric'].metpy.sel(subset)

vtime = datetime.strptime(str(ds.Geopotential_height_isobaric.time.data[0].astype('datetime64[ms]')),
                          '%Y-%m-%dT%H:%M:%S.%f')

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

thta = mpcalc.potential_temperature(lev[:, None, None], tmpk)

relvor = mpcalc.vorticity(uwnd, vwnd)

div = mpcalc.divergence(uwnd, vwnd)

epv = mpcalc.potential_vorticity_baroclinic(thta, lev, uwnd, vwnd)

thta_DT = interpolate_to_isosurface(epv[1:].values*1e6, thta[1:].values, 2)
uwnd_DT = interpolate_to_isosurface(epv[1:].values*1e6, uwnd[1:].metpy.convert_units('kt').values, 2)
vwnd_DT = interpolate_to_isosurface(epv[1:].values*1e6, vwnd[1:].metpy.convert_units('kt').values, 2)

ip850 = list(lev.m).index(850*100.)
ip925 = list(lev.m).index(925*100.)
relvor_925850 = np.average(relvor[[ip925,ip850],:,:], axis=0)

plot_PV(250)

oc_open: server error retrieving url: code=403 message="Request too big=543.20094 Mbytes, max=500.0"oc_open: server error retrieving url: code=403 message="Request too big=543.20094 Mbytes, max=500.0"

KeyboardInterrupt: 

In [ ]:
plot_DT()